Contenido bajo licencia Creative Commons BY 4.0 y código bajo licencia MIT. © Juan Gómez y Nicolás Guarín-Zapata 2020. Este material es parte del curso Modelación computacional en el programa de Ingeniería Civil de la Universidad EAFIT.

# Diseño de una presa trapezoidal de concreto.

## Introducción

El presente Notebook describe un problema de ingeniería civil cuya solución requiere, además del uso de herramientas de simulación numérica, la aplicación de conceptos de la materias Mecánica de los Medios Continuos y Mecánica de sólidos. El proyecto se enmarca en la modalidad de **Aprendizaje Basado en Proyectos** dentro de la cual se expone a los estudiantes a un problema típico de ingeniería, posiblemente sin solución única y en algunas ocasiones mal formulado.

El tema particular del presente proyecto corresponde al diseño de una presa de concreto bajo ciertas restricciones impuestas por las resistencias de los materiales. Como diseño inicial se plantea una presa de forma trapezoidal y para desarrollar el análisis se usa la herramienta por elementos finitos [SolidsPy](https://solidspy.readthedocs.io/en/latest/readme.html). El mayor puntaje lo recibirá la propuesta que obtenga la mayor relación beneficio/costo y que se encuentre acompañada de la mejor justificación física y metodológica.

Inicialmente se define el problema en términos de parámetros de materiales, costos de los materiales, y capacidad de generación de energía.


Posterioromente se describen varias subrutinas que pueden ser útiles para la solución del problema, aunque en general se espera que los equipos de trabajo desarrollen rutinas adicionales de acuerdo a las necesidades del método de análisis. Posteriormente se describen las rutinas de procesado de mallas y de generación de cargas y en la parte final se muestran los pasos mínimos de un análisis por elementos finitos para un caso partícuar.

## Aspectos a considerar

<div class="alert alert-warning">
    
En la calificación de los proyectos se tendrán en cuenta los siguientes aspectos:



1. Organización, claridad y documentación del Notebook (5%).
2. Discusión relativa a la verificación del modelo numérico (5%).
3. Calidad de la solución en términos de la relación beneficio/costo. (40%).
4. Método de solución usado para llegar al diseño óptimo (50%).





</div>


## Definición del problema.

Para atender las demandas de energía de un país se requiere diseñar una presa de concreto de forma trapezoidal (ver figura). Considerando el caudal disponible en la fuente hídrica, la presa debe tener una altura de al menos $100 \text{ m}$ por lo que se requiere diseñar la misma de manera que ofrezca la mayor relación beneficio/costo. Esta relación se define como

$$R=\frac {E_w}C$$

donde $E_w$ es la cantidad de energía producida (medida en megavatios) y $C$ es el costo total (en dolares). La generación de energía incrementa mientras más altura haya en el nivel de agua. Sin embargo, para alturas bajas la eficiencia de generación no son tan buenas. La función que describe la generación es la siguiente

$$E_w = 10000 H_p\left(1 - e^{-H_p/100}\right)\, .$$




En este sentido, el mejor diseño será aquel que genere la mayor cantidad de energía, con el menor volumen de concreto y por ende con el menor costo.

Para el diseño de la presa se deben tener en cuenta las siguientes condiciones.

* El metro cúbico de concreto del tipo asumido como material base tiene un costo de 1000 USD. Para calcular el volumen de concreto use $V_c = 1.0\, A_s$ donde $A_s$ es el área superficial de la presa.

* Cada megapascal de aumento en la resistencia a la compresión del concreto tiene un costo de 100 USD.

* Cada megapascal de aumento en la resistencia a la tensión del concreto tiene un costo de 500 USD.

* Cada megapascal de aumento en la resistencia al corte del concreto tiene un costo de 300 USD.

El material base para el concreto de la presa tiene las siguientes propiedades:

* Módulo de elasticidad: 50 GPa.

* Relación de Poisson: 0.20.

* Resistencia a la compresión: 60 MPa.

* Resistencia a la tensión: 10 MPa.

* Resistencia al corte : 15 MPa.

La presa se construirá sobre un basamento  rocoso (basalto asumido elástico y de extensión infinita) con las siguientes propiedades:

* Módulo de elasticidad: 60 GPa.

* Relación de Poisson: 0.20.

Se asume además que:

* La presa estará sometida a la acción de la presión hidrostática impartida por un fluido de peso especifico $\gamma = 9.8 \text{ kN/m}^3$ y dada por:

$$p = \gamma z$$

donde  $z$ es la distancia desde la superficie libre del fluido asumida en la misma cota de la cresta de la presa.

* El programa de análisis asume condiciones de deformación plana.

* Es posible considerar las fuerzas de cuerpo correspondientes al peso propio de la presa.

<center>
    <img src="img/presa.svg"
         alt="Diagrama de la presa."
         style="width:400px">
</center>

### Dimensiones del modelo

Las dimensiones del modelo son las siguientes:

* $H_p $ : es la altura de la presa .

* $A_p$ : es el ancho de la pata.

* $A_c$ : es el ancho de la cresta.

Los paramétros para el rectangulo que representa el suelo no se consideran paramétros de diseño, pero sus valores pueden afectar la precisión de los resultados. Estos se definen como:

* $H_s$ : profundidad del deposito de suelo.

* $D_l$ : distancia de la frontera izquierda medida desde la pata de la presa.

* $D_r$ : distancia de la fronetra derecha medida desde la pata de la presa.


<center>
    <img src="img/presa_dim.svg"
         alt="Diagrama de la presa."
         style="width:400px">
</center>

## Algunas unidades y equivalencias útiles

* $1 \quad \text{ N} = 1\quad \text{ kg m/s}^2$.

* $1\quad \text{ Pa} = 1 \text{ N/m}^2$.

* $1\quad \text{ kPa} = 1 \times 10^3 \text{ Pa}$.

* $1\quad \text{ MPa} = 1 \times 10^6 \text{ Pa}$.

* $1\quad \text{ GPa} = 1 \times 10^9 \text{ Pa}$.

### Referencias

* Juan Gómez, Nicolás Guarín-Zapata (2018). SolidsPy: 2D-Finite Element Analysis with Python, <https://github.com/AppliedMechanics-EAFIT/SolidsPy>.

In [1]:
from IPython.core.display import HTML
def css_styling():
    styles = open('./nb_style.css', 'r').read()
    return HTML(styles)
css_styling()